In [1]:
import pandas as pd
import requests
from geopy.distance import geodesic


In [2]:
# Load your CSV file
df = pd.read_csv("socio_economic_df (1).csv")

# Ensure your CSV has latitude and longitude for each geoid
# If not, you need a mapping to get those coordinates


In [3]:
def fetch_nearby_facilities(lat, lon, facility_type):
    """Fetch nearby hospitals, clinics, and pharmacies within 5km using OpenStreetMap."""
    overpass_url = "http://overpass-api.de/api/interpreter"
    query = f"""
    [out:json];
    node["amenity"="{facility_type}"](around:5000,{lat},{lon});
    out count;
    """
    response = requests.get(overpass_url, params={"data": query})
    if response.status_code == 200:
        data = response.json()
        return len(data.get("elements", []))  # Count the number of facilities
    return 0


In [ ]:
facility_types = ["hospital", "clinic", "pharmacy"]

df["hospital_count"] = df.apply(lambda row: fetch_nearby_facilities(row["latitude"], row["longitude"], "hospital"), axis=1)
df["clinic_count"] = df.apply(lambda row: fetch_nearby_facilities(row["latitude"], row["longitude"], "clinic"), axis=1)
df["pharmacy_count"] = df.apply(lambda row: fetch_nearby_facilities(row["latitude"], row["longitude"], "pharmacy"), axis=1)

# Save results
df.to_csv("output_with_facility_counts.csv", index=False)
